In [1]:
# prepare data

from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# normalize
x_train, x_test = x_train / 255.0, x_test / 255.0

In [2]:
# https://qiita.com/T-STAR/items/de933c67b52a47f9efef

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Lambda, Activation

def provide_model(optimizer = "adam"):
    num_classes = 10
    input_shape = (32,32,3)
    base_input_shape = (224,224,3)
    model_class = tf.keras.applications.EfficientNetB0

    x = inputs = Input(shape=input_shape)
    #Resize
    x = Lambda(lambda image: tf.image.resize(image, base_input_shape[0:2]), output_shape=base_input_shape)(x)
    #学習済みモデルの組み込み
    base_model = model_class(include_top=False, input_shape=base_input_shape,
    weights='imagenet')
    base_model.trainable = False
    x = base_model(x, training=False)  # trainingをFalseにする
    #トップレイヤーの追加
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    outputs = Activation('softmax')(x)
    model = Model(inputs, outputs)

    output  = Dense(10,activation = "softmax")(x)

    model = Model(inputs, output)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# train

model = provide_model()

model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [ ]:
# eval

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("\n")
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)